<a href="https://colab.research.google.com/github/christine127/Portfolio_Recommendation_Project/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC_%ED%8C%8C%EC%9D%B4%EB%84%90_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install finance-datareader
# !pip install opendartreader

In [3]:
!pip install pykrx

     |████████████████████████████████| 81 kB 4.9 MB/s 
     |████████████████████████████████| 60 kB 7.2 MB/s 
     |████████████████████████████████| 251 kB 26.0 MB/s 


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# import FinanceDataReader as fdr
# import OpenDartReader
import datetime
from pykrx import stock

In [5]:
# api_key = '5729a47b92ddd434526831ee99ff01f65e6434db'
# dart = OpenDartReader(api_key) 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [167]:
# 전체 데이터
# dataset = pd.read_excel('/content/drive/MyDrive/소회의실 프로젝트/Dataset.xlsx')
dataset= pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/전처리_완료/원본_brif_공모수정.csv', encoding ='cp949')
file= dataset.copy()
customer_info = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/전처리_완료/customer_info_brif_공모수정.csv', encoding ='cp949', index_col= 0)

# 10명 샘플 데이터
# file = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/sample10.csv')
# file['고객구분코드'].unique()}

In [169]:
drop_idx = [352,
 1158,
 2250,
 2394,
 2870,
 3273,
 3283,
 3397,
 3592,
 4163,
 4260,
 4531,
 4877,
 4892,
 5340,
 6339,
 6944,
 6992,
 6993,
 7026,
 7335,
 8188,
 8234,
 8341,
 8357,
 8394,
 8596,
 8700,
 10169,
 10995,
 11254,
 11802,
 12625,
 12957,
 12969,
 14272,
 16150,
 16477,
 16621,
 16629,
 16790,
 17579,
 18869,
 18908,
 19004,
 19258,
 19262,
 19551,
 20248,
 20550,
 21114,
 21405,
 21883,
 21960,
 22213,
 22849,
 23048,
 23189,
 23381,
 24022]

In [170]:
drop_id = customer_info.loc[customer_info.index.isin(drop_idx),'고객구분코드']
file_drop_idx = file.loc[file['고객구분코드'].isin(drop_id)].index

In [172]:
# customer_info.drop(drop_idx, axis=0, inplace=True)
customer_info.drop(['상품개수','kosdaq_cap_PER',
       'large_cap_PER', 'medium_cap_PER', 'small_cap_PER', 'kosdaq_cap_PBR',
       'large_cap_PBR', 'medium_cap_PBR', 'small_cap_PBR', 'kosdaq_cap_ROE',
       'large_cap_ROE', 'medium_cap_ROE', 'small_cap_ROE', 'kosdaq_cap_EPS',
       'large_cap_EPS', 'medium_cap_EPS', 'small_cap_EPS', 'kosdaq_cap_DIV',
       'large_cap_DIV', 'medium_cap_DIV', 'small_cap_DIV'], axis=1, inplace= True)

In [179]:
file.drop(['Unnamed: 0','PER', 'PBR', 'ROE', 'EPS', 'DIV','TR', 'ATR_7', 'ATR_14'], axis=1, inplace= True)
file.drop(file_drop_idx, axis=0, inplace= True)

In [186]:
customer_info.set_index('고객구분코드', inplace= True)

In [175]:
## 주문일자
# file['date'] = pd.to_datetime(file['주문일자'],format='%Y%m%d').astype(str)
file[['주문일자']] = file[[ '주문일자']].apply(pd.to_datetime, format='%Y-%m-%d')

In [ ]:
##상품번호
file['code'] = file['상품번호'].str[-6:]

In [17]:
customer_info = pd.DataFrame(file.groupby(['고객구분코드','고객성별구분코드','동일나이군구분코드','주소(시도)'])['상품명'].nunique()).reset_index()
customer_info= customer_info.set_index('고객구분코드')
customer_info =customer_info.rename(columns={'상품명':'상품개수','주소(시도)':'주소'})

In [18]:
customer_info

,고객성별구분코드,동일나이군구분코드,주소,상품개수
고객구분코드,,,,
A00010001,2,30,경북,6
A00010002,2,20,대전,6
A00010004,2,50,대전,1
A00010005,1,20,대구,6
A00010006,1,30,충북,13
...,...,...,...,...
A0009989,2,30,서울,4
A000999,2,30,경기,6
A0009991,1,30,경기,9


### 1. 거래수량 :총 매수수량
### 2. 거래액: 총 매수액


In [19]:
#구매자별 매수금액
customer_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결금액'].sum() 
customer_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결수량'].sum()

# 코드 수정함
customer_info['총매수금액']= customer_purchase
customer_info['총매수수량'] = customer_amount


### 3. 투자상품의 종류별 비중(총매수금액을 가중치로) 
ETF, 대형주, 중형주, 소형주, KOSDAQ 


In [20]:
kosdaq_tickers = stock.get_market_ticker_list("20200120", market="KOSDAQ")
kospi_tickers = stock.get_market_ticker_list("20200120", market="KOSPI")
kospi_caps  = stock.get_market_cap_by_ticker("20200120").loc[kospi_tickers,'시가총액'].sort_values(ascending= False)
kosdaq_caps  = stock.get_market_cap_by_ticker("20200120").loc[kosdaq_tickers,'시가총액'].sort_values(ascending= False)

kosdaq_tickers_2 = stock.get_market_ticker_list("20200331", market="KOSDAQ")
kospi_tickers_2 = stock.get_market_ticker_list("20200331", market="KOSPI")
kospi_caps_2  = stock.get_market_cap_by_ticker("20200331").loc[kospi_tickers_2,'시가총액'].sort_values(ascending= False)
kosdaq_caps_2  = stock.get_market_cap_by_ticker("20200331").loc[kosdaq_tickers_2,'시가총액'].sort_values(ascending= False)
kospi_caps = pd.concat([kospi_caps,kospi_caps_2],axis=1).mean(axis=1)
kosdaq_caps = pd.concat([kosdaq_caps,kosdaq_caps_2],axis=1).mean(axis=1).index


large_cap = kospi_caps[:100].index 
medium_cap = kospi_caps[100:300].index
small_cap = kospi_caps[300:].index


In [21]:
#stock_type
file['stock_type']= ' '
file.loc[file['code'].isin(large_cap), 'stock_type'] ='large_cap'
file.loc[file['code'].isin(medium_cap), 'stock_type'] ='medium_cap'
file.loc[file['code'].isin(small_cap), 'stock_type'] ='small_cap'
file.loc[file['code'].isin(kosdaq_caps), 'stock_type'] ='kosdaq_cap'

In [22]:
#종목타입별 매수금액 비중

#구매자별 매수금액
customer_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결금액'].sum() 
customer_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['총체결수량'].sum()
#구매자별 상품별 매수금액
product_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드','code','stock_type'])['총체결금액'].sum().unstack().fillna(0)
#구매자별 종목타입별 매수금액 비중
type_purchase = product_purchase.groupby('고객구분코드').sum().apply(lambda x: x/customer_purchase )

customer_info = customer_info.join(type_purchase)
customer_info['총매수금액']= customer_purchase
customer_info['총매수수량'] = customer_amount

### 4. 투자한 섹터 개수
- 사용한 외부데이터 출처 : http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201 
주식 - 세부안내 - 업종별 분류현황

In [23]:
sector_kospi_200331 = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/Sector_KOSPI_200331.csv', encoding='cp949')
sector_kosdaq_200331 = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/Sector_KOSDAQ_200331.csv', encoding='cp949')
# new_stock = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/신규상장종목.csv', encoding='cp949')
# new_stock.rename(columns={'업종':'업종명'}, inplace=True)
sector_total = pd.concat([sector_kospi_200331.loc[:, ['종목코드', '업종명']], sector_kosdaq_200331.loc[:, ['종목코드', '업종명']]])
sector_total = sector_total.reset_index(drop = True)

#상장폐지, 신규상장 종목 추가
sector_total=sector_total.append({'종목코드' : '011160' , '업종명' : '건설업'} , ignore_index=True) # 3월 24일에 상장폐지된 코스피 두산건설
sector_total=sector_total.append({'종목코드' : '122800' , '업종명' : '금속'} , ignore_index=True) # 2월 12일에 상장폐지된 코스닥 썬테크놀로지스(썬텍)
sector_total=sector_total.append({'종목코드' : '341160' , '업종명' : '금융'} , ignore_index=True) # 1월 30일에 신규상장된 코스닥 하나금융15호스팩 - 작은 이상한점 : KRX종목리스트에는 이 종목이 안뜨네요. 일단 데이터에 있으니 추가.

In [33]:
sector_total.rename(columns={'종목코드':'code'}, inplace=True)
file = pd.merge(file, sector_total, how='left', left_on='code', right_on='code')

In [34]:
# 투자한 섹터 개수
sector_df = pd.DataFrame(file.groupby(['고객구분코드', '업종명']).size()).reset_index()
sector_df = pd.DataFrame(sector_df.groupby(['고객구분코드'])['업종명'].size()).reset_index()
sector_df.rename(columns={'업종명':'업종개수'}, inplace=True)

In [35]:
customer_info = pd.merge(customer_info, sector_df, on='고객구분코드')
customer_info

,고객구분코드,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,kosdaq_cap,large_cap,medium_cap,small_cap,업종개수
0,A00010001,2,30,경북,6,1295850,28,0.000000,0.633021,0.075665,0.291315,5
1,A00010002,2,20,대전,6,2096450,95,0.390970,0.512676,0.000000,0.096353,6
2,A00010004,2,50,대전,1,5190713,22699,1.000000,0.000000,0.000000,0.000000,1
3,A00010005,1,20,대구,6,1117795,89,0.490206,0.356953,0.000000,0.152841,6
4,A00010006,1,30,충북,13,4536930,188,0.641888,0.339326,0.018786,0.000000,10
...,...,...,...,...,...,...,...,...,...,...,...,...
7993,A0009989,2,30,서울,4,267700,10,0.039895,0.932013,0.028091,0.000000,4
7994,A000999,2,30,경기,6,6004200,795,0.313006,0.000000,0.315213,0.371781,5
7995,A0009991,1,30,경기,9,212329,202,0.844581,0.000000,0.155419,0.000000,9
7996,A0009995,1,30,인천,6,19312230,1030,0.131726,0.825314,0.041922,0.001039,5


### 5. 종목보유기간: 매수-매도 기간 
거래량을 차감해서 계산하는 형태


In [ ]:
# return 2가지
# 1. 매도된 주식이 언제 매수한 주식인지 계산
# 2. 매도가 안되고 남아 있는 주식의 인덱스 
# **** 거래일시가 sorting되어 있어야 작동 **** 
def first_sell_date_idx(df):
    a, b = np.array([]), np.array([])
    for idx, row in df.iterrows():
        if row['매도매수구분코드'] == 2:
            a = np.append(a, np.array(row['총체결수량']))
            b = np.append(b, np.array(0))
        elif row['매도매수구분코드'] == 1:
            b = np.append(b, np.array(row['총체결수량']))
            a = np.append(a, np.array(0))
            
#     print('a: ', a)
#     print('b: ', b)

    idx = np.array([])
    for i in range(len(a)):
        flag = True
        flag2 = True
        if b[i] == 0:
            idx = np.append(idx, np.array([-1]))
        else:
            for j in range(0, i):
                if(b[i] <= a[j]):                  
                    a[j] = a[j] - b[i]
                    b[i] = 0
                    if flag:
                        idx = np.append(idx, np.array([j]))
                        flag = False
                        flag2 = False
                    break
                elif (b[i] > a[j]) & (a[j] !=0):   
                    b[i] = b[i] - a[j]
                    a[j] = 0
                    if flag:
                        idx = np.append(idx, np.array([j]))
                        flag = False
                        flag2 = False
                        
                if((j == i-1) & (b[i] != 0)):
                    if flag2:
                        idx = np.append(idx, np.array([0]))
                    else:
                        idx[-1] = 0
                    b[i] == 0
                    
                
                   
    rest_index = np.where(a != 0)[0].tolist()
    return np.array(idx, dtype = np.int64).tolist(), rest_index

In [ ]:
# 각각의 주문 보유기간 계산 하는 함수
# (고객번호, 계좌번호, 상품번호)별로 묶어서 계산
def buysell(df):
    
    cols = ['고객구분코드', '주문일자', '주문시간대', '상품번호', '상품명', 'stock_type', '매도매수구분코드', '실주문단가', '주문수량', '총체결수량', '총체결금액']
    combine_df = pd.DataFrame(columns= cols)
    
    for userID in tqdm(df['고객구분코드'].unique()):
        
        invest = df[(df['고객구분코드'] == userID)][['고객구분코드', '주문일자', '주문시간대', '상품번호', '상품명', 'stock_type','매도매수구분코드', '실주문단가', '주문수량', '총체결수량', '총체결금액']]

        # 매수매도되지 않은 주식의 보유기간 계산을 위한 기준일
        first_date = min(invest['주문일자'])
        # print(first_date)
        base_date = datetime.datetime(2020, 4, 1, 15, 0, 0)
        end_march = datetime.datetime.strptime('2020-03-15', '%Y-%m-%d')

        # (고객번호, 계좌번호)가 거래한 상품번호
        stock_code = invest['상품번호'].unique().tolist()

        # loop 한번 돌 때 마다 상품번호 끼리 묶음 
        for num in stock_code:
            temp = invest.loc[invest['상품번호'] == num]
            
            temp.loc[-1] =  {'고객구분코드': userID,
                           '주문일자': first_date,
                             '주문시간대':0,
                           '상품번호': num,
                           '매도매수구분코드': 2,
                            '총체결수량': 0}
            temp = temp.sort_values(by=['매도매수구분코드'], ascending=False).sort_values(by=['주문일자', '주문시간대'])

 

            """
            종목보유기간 계산
            - 매도시, 종목 보유 기간이 찍힘
            - 일반적으로 매도매수코드가 2일때(매수) NaN값을 가짐
            - 매도매수코드가 2인데 종목보유기간이 있는 행 --> 실제로 팔리지 않아 기준일(4월1일)에 팔았다고 가정하고 보유기간 계산
            - 3월 말 이후에 매수하고 매도되지않은 주식의 종목보유기간 --> -1 
            - 과거의 매수량보다 매도량이 많을 경우, 고객의 첫 구매일에 구매한 것으로 가정 --> 매수시점 idx = 0
            """
            # 총 매도수량
            sell_quantity = temp[temp['매도매수구분코드']==1]['총체결수량'].sum()
            # 총 매수수량
            buy_quantity = temp[temp['매도매수구분코드']==2]['총체결수량'].sum()

            # print('매도수량: ', sell_quantity, '    매수수량: ', buy_quantity)

            temp['매수시점idx'], rest_index =  first_sell_date_idx(temp)
            temp['종목보유기간'] = np.where(temp['매수시점idx'] == -1, np.nan, 
                                           (temp['주문일자'] - temp.iloc[temp['매수시점idx']]['주문일자'].values).dt.days)
            # 매도되지 못한 나머지
            for idx in rest_index:
                temp.iloc[idx, temp.columns.get_loc('종목보유기간')] = np.where(temp.iloc[idx, temp.columns.get_loc('주문일자')] > end_march, -1, 
                                                                          (base_date - temp.iloc[idx, temp.columns.get_loc('주문일자')]).days)
            
            
            temp.drop(index=temp.index[:1], axis=0, inplace=True)
            temp.drop('매수시점idx', axis=1, inplace=True)
            

            combine_df = pd.concat([combine_df, temp], axis=0)
        
  
    return combine_df        


In [ ]:
def for_grouping(x):   
    if (~x.isin([-1])).sum() == 0: # 모든 종목보유기간이 -1 
        avg = -1
    else:
        avg = x[x >= 0].mean()
   
    return avg
    

# 고객의 평균 보유기간 계산 하는 함수    
def cal_term(df): 
    # cust_acc_set = df['고객구분코드'].unique()
    cust_cap_term = pd.DataFrame(columns=['고객구분코드', 'large_cap', 'medium_cap', 'small_cap', 'kosdaq_cap'])
    # for cust, in cust_acc_set:
        # 매매 개별 종목보유기간 불러오기
    temp = buysell(df)[['고객구분코드', 'stock_type','종목보유기간']]

    # 그룹핑 하고 평균 내기, 모든 종목 보유기간이 -1일 경우 -1로 
    temp = temp[['고객구분코드', 'stock_type', '종목보유기간']]
    grouped = temp.groupby(['고객구분코드', 'stock_type'])
    temp = grouped.agg({'종목보유기간': for_grouping})

    temp = temp.pivot_table(index='고객구분코드', columns='stock_type', values='종목보유기간').reset_index()
    cust_cap_term = pd.concat([cust_cap_term, temp], ignore_index=True)
    cust_cap_term.columns = ['고객구분코드', 'large_cap_period', 'medium_cap_period', 'small_cap_period', 'kosdaq_cap_period']

    cust_cap_term.fillna(0, inplace=True)
    return cust_cap_term
    # -1은 평균값으로 넣기 --> 매수는 했지만 매도는 없는 형태



In [ ]:
pd.options.mode.chained_assignment = None
cal_term_df = cal_term(file)
cal_term_df = cal_term_df.replace(-1, np.nan)
cal_term_df.fillna(cal_term_df[cal_term_df[['large_cap', 'medium_cap', 'small_cap', 'kosdaq_cap']] > 0].mean(), inplace=True)
# cal_term_df

In [ ]:
customer_info = pd.merge(customer_info, cal_term_df, on='고객구분코드')
# customer_info

NameError: ignored

### 6. 수익률: (총매도금액/총매수금액 -1)
매도를 하지 않은 경우의 수익률은 어떻게 계산할지 조금 더 고민→ 4월 1일 매도 기준 

In [ ]:
## 2020년 4월 1일 주가데이터 불러오기
stock_0401 = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/200401전종목가격.csv', encoding='cp949')

In [ ]:
# 수익률 계산 
def cal_rate_of_return(df):
    cust_set = df['고객구분코드'].unique()
    cust_rate_return = pd.DataFrame(columns=['고객구분코드', '수익률'])
    
    for cust in cust_set:
        invest = df[df['고객구분코드'] == cust][['고객구분코드', '주문일자', '상품번호', '상품명', '매도매수구분코드','실주문단가', '주문수량', '총체결수량', '총체결금액']]
        combine_df = pd.DataFrame(columns= invest.columns)

        # (고객번호)가 거래한 상품번호
        stock_code = invest['상품번호'].unique().tolist()
        
        total_sell_price = 0
        total_buy_price = 0
        
        
        # loop 한번 돌 때 마다 상품번호 끼리 묶음 
        for num in stock_code:
            temp = invest.loc[invest['상품번호'] == num]

            first_date = min(temp['주문일자'])
            first_date = first_date.strftime('%Y%m%d')
            
            
            # ticker_code = temp['상품번호'].apply(lambda x: x[6:])
            ticker_code = temp['상품번호'].apply(lambda x: x[6:]).values[0]
            
            # 매도매수수량
            sell_quantity = temp[temp['매도매수구분코드']==1]['총체결수량'].sum()
            buy_quantity = temp[temp['매도매수구분코드']==2]['총체결수량'].sum()
            
            
            buy_price = temp[temp['매도매수구분코드']==2]['총체결금액'].sum()
            sell_price = temp[temp['매도매수구분코드']==1]['총체결금액'].sum()
            
            if buy_quantity >= sell_quantity:
                # 4월 1일 종가 가져오기
                # closing_price = stock.get_market_ohlcv_by_date("20200401", "20200401", ticker_code).iloc[0]['종가']
                closing_price = stock_0401[stock_0401['종목코드']== ticker_code].iloc[0]['종가'] # KRX사이트에서 받은 데이터에서 4월 1일 종가 가져오기
                sell_price = sell_price + (closing_price*(buy_quantity-sell_quantity))
            else:
                # 고객의 첫구매일 종가 가져오기
                # closing_price = stock.get_market_ohlcv_by_date(first_date, first_date, ticker_code).iloc[0]['종가']
                closing_price = df[df['code']== ticker_code].iloc[0]['당일종가'] # pykrx보다는 저희 원래 데이터가 정확하니 원래 데이터에서 종가 가져오는게 좋을것 같음
                buy_price = buy_price + (closing_price*(sell_quantity-buy_quantity))
                
            total_sell_price += sell_price
            total_buy_price += buy_price
        
        # 수익률
        rate_of_return = (total_sell_price/total_buy_price -1) * 100

        cust_rate_return = cust_rate_return.append({'고객구분코드':cust, '수익률':rate_of_return}, ignore_index=True)
    
    return cust_rate_return

In [ ]:
customer_info = pd.merge(customer_info, cal_rate_of_return(file), on='고객구분코드')
# customer_info

,고객구분코드,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,kosdaq_cap,large_cap,medium_cap,small_cap,업종개수,large_cap_period,medium_cap_period,small_cap_period,kosdaq_cap_period,수익률
0,A0001,1,30,경기,149,69911181,25198,0.778486,0.009521,0.024741,0.187252,25,41.666667,23.000000,26.757143,24.744604,-3.406871
1,A00010,1,20,경남,10,725350,366,0.679107,0.000000,0.075550,0.245344,5,0.000000,69.000000,44.750000,38.833333,-19.135678
2,A000100,2,40,서울,10,12100965,602,0.452763,0.289233,0.258003,0.000000,3,42.000000,31.000000,0.000000,43.250000,-15.010250
3,A0001000,1,30,광주,6,1223372,197,0.747271,0.000000,0.000000,0.252729,2,0.000000,0.000000,8.000000,1.000000,-3.898675
4,A00010000,1,20,경기,12,17936233,3683,0.363163,0.515705,0.093949,0.027182,6,15.750000,0.000000,25.000000,16.000000,1.957477
5,A00010002,2,20,대전,6,2096450,95,0.390970,0.512676,0.000000,0.096353,3,28.500000,0.000000,12.000000,11.500000,2.332991
6,A00010003,1,20,울산,7,1135504,83,0.293970,0.534741,0.171290,0.000000,5,19.000000,1.666667,0.000000,14.375000,-5.490870
7,A00010004,2,50,대전,1,5190713,22699,1.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,16.812500,-34.880738
8,A00010005,1,20,대구,6,1117795,89,0.490206,0.356953,0.000000,0.152841,1,23.000000,12.500000,0.000000,0.000000,11.426753
9,A00010006,1,30,충북,13,4536930,188,0.641888,0.339326,0.018786,0.000000,6,25.250000,19.000000,0.000000,19.714286,-3.296298


In [ ]:
customer_info= customer_info.set_index('고객구분코드')

## 상한가하한가 여부

In [ ]:
def limit_price(df):
  df['상한가여부'] = df['상한가여부'].map(dict(Y=1, N=0))
  df['하한가여부'] = df['하한가여부'].map(dict(Y=1, N=0))

  df['상하한가여부'] = df['상한가여부'] | df['하한가여부']

  limit_price_df = pd.DataFrame(df.groupby('고객구분코드')['상하한가여부'].sum()/df.groupby('고객구분코드')['상하한가여부'].count())
  return limit_price_df

In [ ]:
limit_df = limit_price(file)

In [ ]:
limit_df

,상하한가여부
고객구분코드,
A0001,0.060686
A00010,0.000000
A000100,0.000000
A0001000,0.000000
A00010000,0.000000
A00010002,0.000000
A00010003,0.000000
A00010004,0.058824
A00010005,0.400000


In [ ]:
customer_info = customer_info.join(limit_df, on='고객구분코드')

,고객성별구분코드,동일나이군구분코드,주소,상품개수,상하한가여부
고객구분코드,,,,,
A0001,1,30,경기,149,0.060686
A00010,1,20,경남,10,0.000000
A000100,2,40,서울,10,0.000000
A0001000,1,30,광주,6,0.000000
A00010000,1,20,경기,12,0.000000
A00010002,2,20,대전,6,0.000000
A00010003,1,20,울산,7,0.000000
A00010004,2,50,대전,1,0.058824
A00010005,1,20,대구,6,0.400000


In [ ]:
customer_info

,고객성별구분코드,동일나이군구분코드,주소,상품개수,상하한가여부
고객구분코드,,,,,
A0001,1,30,경기,149,0.060686
A00010,1,20,경남,10,0.000000
A000100,2,40,서울,10,0.000000
A0001000,1,30,광주,6,0.000000
A00010000,1,20,경기,12,0.000000
A00010002,2,20,대전,6,0.000000
A00010003,1,20,울산,7,0.000000
A00010004,2,50,대전,1,0.058824
A00010005,1,20,대구,6,0.400000


## PER, PBR, ROE, PSR, PEG
7. PER: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균 
8. PBR: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
9. ROE: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
10. PSR: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
11. PEG: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균

## EPS, DIV
 12. EPS: 대 /중 / 소 / KOSDAQ 각각의 거래액기준 가중평균 - 주당순이익 - 기업 간 실적비교
 13. 배당률(DIV): 대 /중 / 소 / KOSDAQ 각각의 거래액기준 가중평균



In [176]:
jaemu = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/재무정보.csv')

In [177]:
## PER 
PER_df = jaemu.loc[:, ['날짜', '종목코드', 'PER']]
PER_df.rename(columns={'날짜':'date'}, inplace=True)
PER_df.rename(columns={'종목코드':'code'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, PER_df, how='left',on=['code', 'date'])

file['PER'].fillna(0, inplace=True)

In [180]:
## PBR 
PBR_df = jaemu.loc[:, ['날짜', '종목코드', 'PBR']]
PBR_df.rename(columns={'날짜':'date'}, inplace=True)
PBR_df.rename(columns={'종목코드':'code'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, PBR_df, how='left',on=['code', 'date'])

file['PBR'].fillna(0, inplace=True)

In [181]:
## ROE

file['ROE'] = file['PER']/ file['PBR']

file['ROE'].fillna(0, inplace=True)

In [182]:
## EPS 
EPS_df = jaemu.loc[:, ['날짜', '종목코드', 'EPS']]
EPS_df.rename(columns={'날짜':'date'}, inplace=True)
EPS_df.rename(columns={'종목코드':'code'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, EPS_df, how='left',on=['code', 'date'])

file['EPS'].fillna(0, inplace=True)

In [183]:
## 배당률(DIV)
DIV_df = jaemu.loc[:, ['날짜', '종목코드', '배당수익률']]
DIV_df.rename(columns={'날짜':'date'}, inplace=True)
DIV_df.rename(columns={'종목코드':'code'}, inplace=True)
DIV_df.rename(columns={'배당수익률':'DIV'}, inplace=True)

# 원 데이터와 조인
file = pd.merge(file, DIV_df, how='left',on=['code', 'date'])

file['DIV'].fillna(0, inplace=True)

In [187]:
product_purchase_rate = product_purchase.apply(lambda x: x/customer_purchase )

for col in ['PER', 'PBR', 'ROE', 'EPS', 'DIV']:
  #고객의 상품별 EPS, DIV 평균
  product_col = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드','code','stock_type'])[col].mean().unstack().fillna(0)


  #고객별 종목타입별 EPS, DIV 가중평균
  col_rate = product_col* product_purchase_rate
  type_col = col_rate.groupby('고객구분코드').sum()
  col_mean = pd.Series(np.sum(type_col* type_purchase, axis=1))
  # print(col_mean)


  # customer_info = customer_info.join(col_mean, on='고객구분코드', rsuffix='_'+col)
  customer_info[col]= col_mean

In [ ]:
# ## PEG -- EPS가 (-)일 때 0으로 기록되어 있어서 직접 EPS부터 구해야 함
# stock.get_market_fundamental_by_date(file['주문일시'][38],file['주문일시'][38], file['code'][38])['EPS']
# stock.get_market_fundamental_by_date(file['주문일시'][38]-datetime.timedelta(weeks = 156),file['주문일시'][38]-datetime.timedelta(weeks = 156), file['code'][38])['EPS']
# # dart.finstate(file['상품번호'][38], '2020','11011')

### 14. 외국인 매매, 기관매매와의 매도/매수 일치 횟수 (%)

In [ ]:
#지분율
# stock.get_exhaustion_rates_of_foreign_investment_by_date(file['주문일시'][38],file['주문일시'][38], file['상품번호'][38])
# help(stock.get_exhaustion_rates_of_foreign_investment_by_date)
# stock.get_exhaustion_rates_of_foreign_investment_by_date("20200120", "20200120", "005930")

In [ ]:
foreign = pd.read_csv("/content/drive/MyDrive/소회의실 프로젝트/외부데이터/외국인_순매수.csv")
foreign['날짜'] = foreign['날짜'].apply(pd.to_datetime, format('%Y-%m-%d'))
foreign.rename(columns={'거래대금_순매수':'외국인_순매수액'}, inplace= True)

inst = pd.read_csv("/content/drive/MyDrive/소회의실 프로젝트/외부데이터/기관_순매수.csv")
inst['날짜'] = inst['날짜'].apply(pd.to_datetime, format('%Y-%m-%d'))
inst.rename(columns={'거래대금_순매수':'기관_순매수액'}, inplace= True)

file = pd.merge(file, foreign, how='left', left_on=['주문일자','code'], right_on=['날짜','종목코드']).drop(['날짜','종목코드','종목명'], axis=1)
file = pd.merge(file, inst, how='left', left_on=['주문일자','code'], right_on=['날짜','종목코드']).drop(['날짜','종목코드','종목명'], axis=1)


In [ ]:
file['외국인_순매수액'] = file['외국인_순매수액'].fillna(-1)
file['기관_순매수액'] = file['기관_순매수액'].fillna(-1)

file['외국인_매도매수구분코드'] = file['외국인_순매수액'].apply(lambda x: 1 if x<0 else (2 if x>0 else 0))
file['기관_매도매수구분코드'] = file['기관_순매수액'].apply(lambda x: 1 if x<0 else (2 if x>0 else 0))

# customer_info = customer_info.set_index('고객구분코드')
customer_info['매도매수일치_외국인'] = file.loc[file['외국인_매도매수구분코드'] ==file['매도매수구분코드'],['고객구분코드','매도매수구분코드']].groupby('고객구분코드').count().apply(lambda x: x/file.groupby('고객구분코드')['매도매수구분코드'].count()).fillna(0)
customer_info['매도매수일치_기관'] = file.loc[file['기관_매도매수구분코드'] ==file['매도매수구분코드'],['고객구분코드','매도매수구분코드']].groupby('고객구분코드').count().apply(lambda x: x/file.groupby('고객구분코드')['매도매수구분코드'].count()).fillna(0)

### ATR

In [189]:
ohlcv =pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/ohlcv_ATR.csv')

In [190]:
ohlcv['날짜'] =ohlcv['날짜'].apply(pd.to_datetime, format='%Y-%m-%d')
ohlcv_ATR = ohlcv[['종목코드','날짜','TR','ATR_7','ATR_14']].rename(columns = {'종목코드':'code','날짜':'주문일자'})
file = pd.merge(file, ohlcv_ATR, how='left',on = ['code','주문일자'])

In [191]:
customer_info.drop(['kosdaq_cap_TR','large_cap_TR', 'medium_cap_TR', 'small_cap_TR','kosdaq_cap_ATR_14', 'large_cap_ATR_14', 'medium_cap_ATR_14',
       'small_cap_ATR_14','kosdaq_cap_ATR_7',
       'large_cap_ATR_7', 'medium_cap_ATR_7', 'small_cap_ATR_7'], axis=1, inplace= True)

In [193]:
# customer_info.set_index('고객구분코드',inplace=True)

In [194]:
for col in ['ATR_7']:
  #고객의 상품별 TR(ATR) 평균
  product_col = file.groupby(['고객구분코드','code','stock_type'])[col].mean().unstack().fillna(0)
  
  #고객별 종목타입별 TR(ATR) 가중평균
  col_rate = product_col* product_purchase_rate
  type_col = col_rate.groupby('고객구분코드').sum()

  col_mean = pd.Series(np.sum(type_col* type_purchase, axis=1))

  # customer_info = customer_info.join(col_mean, on='고객구분코드', rsuffix='_'+col)
  customer_info[col]= col_mean

In [195]:
customer_info

,고객성별구분코드,동일나이군구분코드,주소,총매수금액,총매수수량,kosdaq_cap,large_cap,medium_cap,small_cap,large_cap_period,medium_cap_period,small_cap_period,kosdaq_cap_period,수익률,업종개수,상하한가여부,매도매수일치_외국인,매도매수일치_기관,시간대_1,시간대_2,시간대_3,시간대_4,new_stock_rate,PER,PBR,ROE,EPS,DIV,ATR_7
고객구분코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A000973,2,30,경기,1691500.0,19.0,0.000000,1.000000,0.000000,0.000000,5.5,0.000000,0.0,0.000000,-2.483187,2,0.000000,0.250000,0.250000,0.750000,0.125000,0.125000,0.0,0.0,7.588331,0.703141,14.978474,13138.000591,1.907657,6845.289473
A00013780,2,30,서울,18044560.0,588.0,0.634889,0.170689,0.194422,0.000000,0.0,11.500000,0.0,5.600000,-2.593365,3,0.350000,0.350000,0.400000,0.400000,0.350000,0.250000,0.0,0.0,92.113722,6.286813,9.231516,293.946701,0.021560,4684.775463
A0009975,2,50,충북,195560.0,23.0,0.000000,0.000000,1.000000,0.000000,0.0,25.933333,0.0,0.000000,-14.143997,1,0.000000,0.100000,0.200000,0.600000,0.200000,0.200000,0.0,0.0,1.396000,0.696500,2.005266,6032.000000,3.646500,515.642857
A00012551,1,30,경기,20627310.0,4558.0,0.842563,0.000000,0.000000,0.157437,0.0,0.000000,0.0,9.090909,2.291601,15,0.035714,0.464286,0.464286,0.321429,0.357143,0.321429,0.0,0.0,15.819576,1.239198,13.947398,458.046274,0.908531,767.036933
A000706,1,20,경기,1128091.0,1797.0,0.327540,0.324974,0.000000,0.347486,0.0,0.000000,0.6,29.000000,2.535725,6,0.000000,0.454545,0.545455,0.000000,0.181818,0.818182,0.0,0.0,1.965408,0.393823,1.560131,688.495998,0.246067,143.649599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A00011326,1,30,울산,7080700.0,5113.0,0.368975,0.623907,0.007118,0.000000,19.0,0.000000,0.0,12.000000,8.876954,5,0.090909,0.272727,0.454545,0.181818,0.545455,0.272727,0.0,0.0,10.889823,0.465652,14.763757,1719.751414,1.252166,2465.581779
A000162,1,20,전남,94244200.0,5233.0,0.822622,0.036484,0.140894,0.000000,16.0,12.200000,0.0,9.736842,2.416450,6,0.111111,0.481481,0.500000,0.462963,0.296296,0.240741,0.0,0.0,14.381015,3.729991,4.680253,168.219798,0.215730,3327.489259
A0004156,1,20,경기,528900.0,9.0,0.075440,0.924560,0.000000,0.000000,0.0,0.000000,0.0,1.000000,-7.139719,2,0.000000,0.333333,0.166667,0.333333,0.500000,0.166667,0.0,0.0,13.993953,0.677143,16.671019,7119.931917,3.832914,6392.283000


In [196]:
customer_info.reset_index(inplace= True)
customer_info.to_csv('/content/drive/MyDrive/소회의실 프로젝트/전처리_완료/customer_info_brif_하나로수정.csv', index= False)


14. 계좌개수: 계좌개수 
15. 시간대: ? -> 나중에 plot 시각화로 판단

18. 예약주문 횟수
19. 상한가,하한가 여부 횟수: 
주식을 산 날의 변동성을 보기위한 지표 

 
20. 정정/취소 횟수
21. 공모주 비중: 거래액 기준



## 시간대

In [ ]:
customer_info = customer_info.set_index('고객구분코드')

In [ ]:
time_percent = file.groupby('고객구분코드')['주문시간대'].value_counts().unstack().fillna(0).apply(lambda x: x/np.sum(file.groupby('고객구분코드')['주문시간대'].value_counts().unstack().fillna(0), axis=1))
customer_info = customer_info.join(time_percent,on='고객구분코드',lsuffix='시간대_').rename(columns=dict((i,'시간대_'+str(i)) for i in range(1,5)))
customer_info.join(time_percent,on='고객구분코드')

# 공모주 매수 비율

In [ ]:
new_stock = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/외부데이터/신규상장종목.csv', encoding='cp949')

In [ ]:
new_stock['신규종목여부'] = 1

In [ ]:
new_stock_df = new_stock.loc[:, ['종목코드', '신규종목여부']]
new_stock_df.rename(columns={'종목코드':'고객구분코드'}, inplace=True)
new_stock_df['고객구분코드'] = new_stock_df['고객구분코드'].astype(str)
file = pd.merge(file, new_stock_df, on='고객구분코드', how='left')
file['신규종목여부'].fillna(0, inplace=True)

In [ ]:
customer_amount = pd.DataFrame(customer_amount).reset_index()

In [ ]:
gongmo_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객구분코드'])['신규종목여부'].sum()
gongmo_amount = pd.DataFrame(gongmo_amount).reset_index()
gongmo_amount['new_stock_rate'] = gongmo_amount['신규종목여부'] / customer_amount['총체결수량']
gongmo_amount = gongmo_amount.set_index('고객구분코드')

In [ ]:
customer_info = customer_info.join(gongmo_amount['new_stock_rate'], on='고객구분코드' )

,고객성별구분코드,동일나이군구분코드,주소,상품개수,new_stock_rate
고객구분코드,,,,,
A0001,1,30,경기,149,0.0
A00010,1,20,경남,10,0.0
A000100,2,40,서울,10,0.0
A0001000,1,30,광주,6,0.0
A00010000,1,20,경기,12,0.0
A00010002,2,20,대전,6,0.0
A00010003,1,20,울산,7,0.0
A00010004,2,50,대전,1,0.0
A00010005,1,20,대구,6,0.0
